In [2]:
import pandas as pd
train_data = pd.read_csv('Data/bank-additional-full.csv', sep=';')
train_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [3]:
train_data.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

由于Apriori是处理属性名之间的联系，所以布尔类的重名不经处理会导致错误。

In [4]:
train_data['default'] = train_data['default'].map({'yes': 'ydefault', 'no': 'ndefault', 'unknown': 'unknowndefault'})
train_data['housing'] = train_data['housing'].map({'yes': 'yhousing', 'no': 'nhousing', 'unknown': 'unknownhousing'})
train_data['loan'] = train_data['loan'].map({'yes': 'yloan', 'no': 'nloan', 'unknown': 'unknownloan'})
train_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,ndefault,nhousing,nloan,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknowndefault,nhousing,nloan,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,ndefault,yhousing,nloan,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,ndefault,nhousing,nloan,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,ndefault,nhousing,yloan,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


#### 年龄预处理

其次Aprioi本事上还是用于处理分类属性，所以需要将age这一连续属性进行一个转化

In [5]:
train_data.loc[train_data['age'] <= 16, 'age'] = 0
train_data.loc[(train_data['age'] > 16) & (train_data['age'] <= 32), 'age'] = 1
train_data.loc[(train_data['age'] > 32) & (train_data['age'] <= 48), 'age'] = 2
train_data.loc[(train_data['age'] > 48) & (train_data['age'] <= 64), 'age'] = 3
train_data.loc[train_data['age'] > 64, 'age'] = 4
train_data['age'] = train_data['age'].astype(int)
train_data['age'] = train_data['age'].map({0: 'age0', 1: 'age1', 2: 'age2', 3: 'age3', 4: 'age4'})
train_data['age'].value_counts()

age2    21118
age1    11176
age3     8231
age4      663
Name: age, dtype: int64

#### 职业预处理

同样的，由于职业过多，并且职业之间的差别对发放贷款的影响没有有无稳定职业对发放贷款的影响下大，所以我选择将职业值分为两类，一类为有稳定工作，一类为无稳定工作，具体处理过程如下：

In [6]:
JobDict = {}
JobDict['admin.'] = 'HasJob'
JobDict['blue-collar'] = 'HasJob'
JobDict['entrepreneur'] = 'HasJob'
JobDict['housemaid'] = 'HasnotJob'
JobDict['management'] = 'HasJob'
JobDict['retired'] = 'HasnotJob'
JobDict['self-employed'] = 'HasJob'
JobDict['services'] = 'HasJob'
JobDict['student'] = 'HasnotJob'
JobDict['technician'] = 'HasJob'
JobDict['unemployed'] = 'HasnotJob'
JobDict['unknown'] = 'JobUnknown'
train_data['job'] = train_data['job'].map(JobDict)
train_data['job'].value_counts()

HasJob        36189
HasnotJob      4669
JobUnknown      330
Name: job, dtype: int64

#### 教育水平预处理

教育水平数据同样不需要原数据集这样细化，我将它们重新分类为文盲、基础教育、高中教育、高等教育、职业教育这几个类型

In [7]:
EducationDict = {}
EducationDict['basic.4y'] = 'BasicEducation'
EducationDict['basic.6y'] = 'BasicEducation'
EducationDict['basic.9y'] = 'BasicEducation'
EducationDict['high.school'] = 'HighEducation'
EducationDict['illiterate'] = 'Illiterate'
EducationDict['professional.course'] = 'ProfessionalEducation'
EducationDict['university.degree'] = 'UniversityEducation'
EducationDict['unknown'] = 'EducationUnknown'
train_data['education'] = train_data['education'].map(EducationDict)
train_data['education'].value_counts()

BasicEducation           12513
UniversityEducation      12168
HighEducation             9515
ProfessionalEducation     5243
EducationUnknown          1731
Illiterate                  18
Name: education, dtype: int64

### 应用Aprioi算法

经过上述处理并且除去一些不重要或者此次实验中不关心的属性之后，数据集变为：

In [8]:
train_data_categrical = train_data.select_dtypes(include=['object'])
train_data_categrical.drop(['contact', 'month', 'day_of_week', 'poutcome'], axis=1, inplace=True)
train_data_categrical.head()

,age,job,marital,education,default,housing,loan,y
0,age3,HasnotJob,married,BasicEducation,ndefault,nhousing,nloan,no
1,age3,HasJob,married,HighEducation,unknowndefault,nhousing,nloan,no
2,age2,HasJob,married,HighEducation,ndefault,yhousing,nloan,no
3,age2,HasJob,married,BasicEducation,ndefault,nhousing,nloan,no
4,age3,HasJob,married,HighEducation,ndefault,nhousing,yloan,no


不认购定期存款的人群：

In [9]:
train_data_categrical_y = train_data_categrical.loc[train_data['y'] == 'yes']
train_data_categrical_y.drop(['y'], axis=1, inplace=True)
train_data_categrical_y

C:\Users\chris\AppData\Local\Temp\ipykernel_4988\2801054380.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_categrical_y.drop(['y'], axis=1, inplace=True)


,age,job,marital,education,default,housing,loan
75,age2,HasJob,divorced,BasicEducation,unknowndefault,yhousing,nloan
83,age3,HasJob,married,UniversityEducation,unknowndefault,yhousing,nloan
88,age3,HasJob,married,BasicEducation,ndefault,nhousing,nloan
129,age2,HasJob,married,ProfessionalEducation,unknowndefault,yhousing,nloan
139,age2,HasJob,married,BasicEducation,unknowndefault,yhousing,nloan
...,...,...,...,...,...,...,...
41174,age3,HasnotJob,married,UniversityEducation,ndefault,yhousing,nloan
41178,age3,HasnotJob,married,UniversityEducation,ndefault,nhousing,nloan
41181,age2,HasJob,married,UniversityEducation,ndefault,yhousing,nloan
41183,age4,HasnotJob,married,ProfessionalEducation,ndefault,yhousing,nloan


In [10]:
train_data_categrical_n = train_data_categrical.loc[train_data['y'] == 'no']
train_data_categrical_n.drop(['y'], axis=1, inplace=True)
train_data_categrical_n

C:\Users\chris\AppData\Local\Temp\ipykernel_4988\3118438323.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_categrical_n.drop(['y'], axis=1, inplace=True)


,age,job,marital,education,default,housing,loan
0,age3,HasnotJob,married,BasicEducation,ndefault,nhousing,nloan
1,age3,HasJob,married,HighEducation,unknowndefault,nhousing,nloan
2,age2,HasJob,married,HighEducation,ndefault,yhousing,nloan
3,age2,HasJob,married,BasicEducation,ndefault,nhousing,nloan
4,age3,HasJob,married,HighEducation,ndefault,nhousing,yloan
...,...,...,...,...,...,...,...
41180,age2,HasJob,married,UniversityEducation,ndefault,nhousing,nloan
41182,age1,HasnotJob,single,BasicEducation,ndefault,yhousing,nloan
41184,age2,HasJob,married,ProfessionalEducation,ndefault,nhousing,nloan
41185,age3,HasnotJob,married,UniversityEducation,ndefault,yhousing,nloan


### 验证自行实现算法的正确性

首先使用一个简单的例子来验证算法的正确性，以课件上的图为例

In [11]:
from Aprioi import Aprioi
itemSetList2 = [
    ['A', 'C', 'D'],
    ['B', 'C', 'E'],
    ['A', 'B', 'C', 'E'],
    ['B', 'E']
]

tree = Aprioi(itemSetList2, 0.5, 0.5)
freqItemSet, associationRules = tree.apriori()
print('Frequent Itemsets:')
for key, value in freqItemSet.items():
    print('  {}: {}'.format(key, value))

print('\nAssociation Rules:')
for rule in associationRules:
    print('  Rule: ', rule[0], ' -> ', rule[1], ' Confidence: ', rule[2])

Frequent Itemsets:
  1: {frozenset({'E'}), frozenset({'C'}), frozenset({'B'}), frozenset({'A'})}
  2: {frozenset({'C', 'B'}), frozenset({'C', 'E'}), frozenset({'A', 'C'}), frozenset({'B', 'E'})}
  3: {frozenset({'C', 'B', 'E'})}

Association Rules:
  Rule:  {'C'}  ->  {'B'}  Confidence:  0.6666666666666666
  Rule:  {'B'}  ->  {'C'}  Confidence:  0.6666666666666666
  Rule:  {'C'}  ->  {'E'}  Confidence:  0.6666666666666666
  Rule:  {'E'}  ->  {'C'}  Confidence:  0.6666666666666666
  Rule:  {'A'}  ->  {'C'}  Confidence:  1.0
  Rule:  {'C'}  ->  {'A'}  Confidence:  0.6666666666666666
  Rule:  {'B'}  ->  {'E'}  Confidence:  1.0
  Rule:  {'E'}  ->  {'B'}  Confidence:  1.0
  Rule:  {'C'}  ->  {'B', 'E'}  Confidence:  0.6666666666666666
  Rule:  {'B'}  ->  {'C', 'E'}  Confidence:  0.6666666666666666
  Rule:  {'E'}  ->  {'C', 'B'}  Confidence:  0.6666666666666666
  Rule:  {'C', 'B'}  ->  {'E'}  Confidence:  1.0
  Rule:  {'C', 'E'}  ->  {'B'}  Confidence:  1.0
  Rule:  {'B', 'E'}  ->  {'C'}  Co

将Aprioi算法用在没有认购定期存款的数据子集上，代码与结果为：

In [12]:
from Aprioi import Aprioi

itemSetList = []   
for i in range(0, len(train_data_categrical_n)):
    itemSetList.append(set(train_data_categrical_n.iloc[i]))

tree = Aprioi(itemSetList, 0.5, 0.5)
freqItemSet, associationRules = tree.apriori()
# print the frequent itemsets
print('Frequent Itemsets:')
for key, value in freqItemSet.items():
    print('  {}: {}'.format(key, value))

print('\nAssociation Rules:')
for rule in associationRules:
    print('  Rule: ', rule[0], ' -> ', rule[1], ' Confidence: ', rule[2])

Frequent Itemsets:
  1: {frozenset({'married'}), frozenset({'HasJob'}), frozenset({'age2'}), frozenset({'nloan'}), frozenset({'yhousing'}), frozenset({'ndefault'})}
  2: {frozenset({'married', 'HasJob'}), frozenset({'HasJob', 'ndefault'}), frozenset({'nloan', 'married'}), frozenset({'nloan', 'HasJob'}), frozenset({'nloan', 'ndefault'})}
  3: {frozenset({'nloan', 'HasJob', 'ndefault'})}

Association Rules:
  Rule:  {'married'}  ->  {'HasJob'}  Confidence:  0.8907394177531702
  Rule:  {'HasJob'}  ->  {'married'}  Confidence:  0.6129666615455523
  Rule:  {'HasJob'}  ->  {'ndefault'}  Confidence:  0.7839299431556307
  Rule:  {'ndefault'}  ->  {'HasJob'}  Confidence:  0.8986298474868797
  Rule:  {'nloan'}  ->  {'married'}  Confidence:  0.6135880398671096
  Rule:  {'married'}  ->  {'nloan'}  Confidence:  0.824656188605108
  Rule:  {'nloan'}  ->  {'HasJob'}  Confidence:  0.8899335548172758
  Rule:  {'HasJob'}  ->  {'nloan'}  Confidence:  0.8230757412813028
  Rule:  {'nloan'}  ->  {'ndefault'}

对于认购了定期存款的数据集来说，结果类似

In [13]:
itemSetList = []   
for i in range(0, len(train_data_categrical_y)):
    itemSetList.append(set(train_data_categrical_y.iloc[i]))

tree = Aprioi(itemSetList, 0.5, 0.5)
freqItemSet, associationRules = tree.apriori()
# print the frequent itemsets
print('Frequent Itemsets:')
for key, value in freqItemSet.items():
    print('  {}: {}'.format(key, value))

print('\nAssociation Rules:')
for rule in associationRules:
    print('  Rule: ', rule[0], ' -> ', rule[1], ' Confidence: ', rule[2])

Frequent Itemsets:
  1: {frozenset({'married'}), frozenset({'HasJob'}), frozenset({'nloan'}), frozenset({'yhousing'}), frozenset({'ndefault'})}
  2: {frozenset({'HasJob', 'ndefault'}), frozenset({'nloan', 'HasJob'}), frozenset({'nloan', 'ndefault'})}
  3: {frozenset({'nloan', 'HasJob', 'ndefault'})}

Association Rules:
  Rule:  {'HasJob'}  ->  {'ndefault'}  Confidence:  0.9006586169045006
  Rule:  {'ndefault'}  ->  {'HasJob'}  Confidence:  0.781987133666905
  Rule:  {'nloan'}  ->  {'HasJob'}  Confidence:  0.788051948051948
  Rule:  {'HasJob'}  ->  {'nloan'}  Confidence:  0.8326015367727772
  Rule:  {'nloan'}  ->  {'ndefault'}  Confidence:  0.9033766233766234
  Rule:  {'ndefault'}  ->  {'nloan'}  Confidence:  0.8286871574934477
  Rule:  {'nloan'}  ->  {'HasJob', 'ndefault'}  Confidence:  0.708051948051948
  Rule:  {'HasJob'}  ->  {'nloan', 'ndefault'}  Confidence:  0.74807903402854
  Rule:  {'ndefault'}  ->  {'nloan', 'HasJob'}  Confidence:  0.6495115558732428
  Rule:  {'nloan', 'HasJob